In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
GHCOMP_FOLDER = os.path.join(parent_dir, "src","aixd_ara","components")
OUTPUT_PATH = os.path.join(parent_dir, "docs")


In [ ]:
import json

metadata_list = []

skip_subfolders = ["aixd_", "ghuser"]

for folder_name in os.listdir(GHCOMP_FOLDER):
    if folder_name in skip_subfolders: continue
    folder_path = os.path.join(GHCOMP_FOLDER, folder_name)
    if os.path.isdir(folder_path):
        metadata_file = os.path.join(folder_path, 'metadata.json')
        if os.path.isfile(metadata_file):
            with open(metadata_file, 'r') as file:
                metadata = json.load(file)
                metadata_list.append([folder_path,metadata])

metadata_list


In [ ]:
import os
import shutil

new_icons_folder = os.path.join(OUTPUT_PATH, "_images", "icons")

if not os.path.exists(new_icons_folder):
    os.makedirs(new_icons_folder)

def copy_icon(folder_path):
    orig_icon_path = os.path.join(folder_path, 'icon.png')
    new_icon_name = f"{os.path.basename(folder_path)}.png"
    new_icon_path = os.path.join(new_icons_folder, new_icon_name)
    shutil.copy(orig_icon_path, new_icon_path)
    return new_icon_name


check also:

https://bashtage.github.io/sphinx-material/pymethod.html

In [ ]:
def rst_header1(txt):
    return f"{txt}\n{'=' * len(txt)}\n"


def rst_header2(txt):
    return f"{txt}\n{'-' * len(txt)}\n"


def rst_header3(txt):
    return f"{txt}\n{'^' * len(txt)}\n"


# def rst_header4(txt):
#     return f"{txt}\n{'' * len(txt)}\n"


# def rst_miniheader(txt):
#     return f"\n**{txt}**\n"


def rst_description(text):
    return f"{text}\n\n"


def rst_input_variable(item):
    name = item["name"]
    description = item["description"]
    type = item["typeHintID"]
    accesstype = item["scriptParamAccess"]

    # t= f"{name}\n"
    # t+=f"\t{rst_format_type(type,accesstype)}"
    # t+=f"\t{description}\n"

    t = f"- **{name}** {rst_format_type(type,accesstype)} -- {description}\n"
    return t


def rst_output_variable(item):
    name = item["name"]
    description = item["description"]

    # t= f"{name}\n"
    # t+=f"  {description}\n"

    t = f"- **{name}** -- {description}\n"
    return t


def rst_format_type(type, accesstype):
    if accesstype == 0:
        x = f"({type})"
    if accesstype == 1:
        x = f"[List of ({type})]"
    if accesstype == 2:
        x = f"[Tree of ({type})]"
    # return f"``{x}``" #code formatting
    return f"*{x}*"  # italic formatting


def rst_icon(path):
    t = f".. image:: _images/icons/{path}\n"
    t += f"\t:align: left\n"
    t += f"\t:height: 24\n"
    t += f"\t:width: 24\n"
    t += "\n"
    return t


def process_one_metadata(path, metadata):
    t = ""
    t += rst_header2(metadata["name"])

    new_icon_name = copy_icon(path)
    t += rst_icon(os.path.join(new_icon_name))
    t += rst_description(metadata["description"])

    if metadata["ghpython"]["inputParameters"] != []:
        t += "\n**Inputs**\n\n"
        for item in metadata["ghpython"]["inputParameters"]:
            t += rst_input_variable(item)

    if metadata["ghpython"]["outputParameters"] != []:
        t += "\n**Outputs**\n\n"
        for item in metadata["ghpython"]["outputParameters"]:
            t += rst_output_variable(item)

    t += "\n"
    return t

In [ ]:
rst_all=""

rst_all+= ".. _documentation:\n\n*************\nDocumentation\n*************\n\n"

rst_all+= "This is the documentation of the Grasshopper components in **ARA**. \n\n"
rst_all+= "The documentation of **AIXD: AI-eXtended Design** tookit can be found `here <https://aixd.ethz.ch/docs/api.html>`_. \n\n"

for path,metadata in metadata_list:
    rst_all+= process_one_metadata(path,metadata)


print(rst_all)

In [ ]:

with open(os.path.join(OUTPUT_PATH,'documentation.rst'), 'w') as file:
    file.write(rst_all)
    print(file)
